In [36]:
import random
import requests
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objects as go
import plotly.io as pio
import warnings
warnings.filterwarnings("default")


In [3]:
data=pd.read_csv("./data.csv")
data=data.drop(['Address'],axis=1)
data.head(2)

,Name,Age,gender,Dob,City,State,Country,email,IP Address,time of access (YYYY-MM-DDTHH-MM-SS),Browser Detail
0,Shivam Dheer,20,Male,01/04/02,Delhi,Delhi,India,shivamdheer2002@gmail.com,205.150.141.106,2023-02-10T10:29:00,Google Chrome
1,Kaustubh Mishra,20,M,25/10/02,Vellore,Tamil Nadu,India,kmishra2510@gmail.com,48.8.17.204,2023-02-10T9:59:00,Firefox


In [4]:
"""d=request.json"""

'd=request.json'

In [5]:
data['gender']=data['gender'].replace(['Male'],['M'])
data['gender']=data['gender'].replace(['Male '],['M'])
data['gender']=data['gender'].replace(['Female'],['F'])
data.head(2)

,Name,Age,gender,Dob,City,State,Country,email,IP Address,time of access (YYYY-MM-DDTHH-MM-SS),Browser Detail
0,Shivam Dheer,20,M,01/04/02,Delhi,Delhi,India,shivamdheer2002@gmail.com,205.150.141.106,2023-02-10T10:29:00,Google Chrome
1,Kaustubh Mishra,20,M,25/10/02,Vellore,Tamil Nadu,India,kmishra2510@gmail.com,48.8.17.204,2023-02-10T9:59:00,Firefox


In [6]:
for i in range(len(data['Browser Detail'])):
    if(("Google" in data['Browser Detail'][i]) or ("Chrome" in data['Browser Detail'][i])or ("google" in data['Browser Detail'][i])or ("chrome" in data['Browser Detail'][i])):
        data['Browser Detail'][i]="Chrome"
    elif(("Firefox" in data['Browser Detail'][i]) or ("firefox" in data['Browser Detail'][i])):
        data['Browser Detail'][i]="Firefox"
    elif(("Edge" in data['Browser Detail'][i]) or ("edge" in data['Browser Detail'][i])):
        data['Browser Detail'][i]="Edge"
    elif(("Safari" in data['Browser Detail'][i]) or ("safari" in data['Browser Detail'][i])):
        data['Browser Detail'][i]="Safari"
    elif(("Opera" in data['Browser Detail'][i]) or ("opera" in data['Browser Detail'][i])):
        data['Browser Detail'][i]="Opera"

In [7]:
label_encoder =LabelEncoder()
columns=['Name','gender','City','State','Country','email','Browser Detail','Dob','IP Address','time of access (YYYY-MM-DDTHH-MM-SS)']
for item in columns:
    data[item] = label_encoder.fit_transform(data[item])
data.head() 

,Name,Age,gender,Dob,City,State,Country,email,IP Address,time of access (YYYY-MM-DDTHH-MM-SS),Browser Detail
0,7,20,1,0,1,0,0,9,4,1,0
1,3,20,1,9,7,5,0,5,7,6,2
2,1,18,1,4,5,3,0,1,0,2,0
3,8,20,1,2,0,1,0,4,9,2,0
4,2,21,1,8,4,8,0,2,2,4,1


In [30]:
std=StandardScaler()
data[['Age']]=std.fit_transform(data[['Age']])

In [65]:
pca=PCA(n_components=2,random_state=42)
data_pca=pca.fit_transform(data)

In [66]:
data_pca

array([[ 7.83748187, -0.38792706],
       [-3.58812661, -0.59383578],
       [-4.02377498, -2.45987272],
       [ 6.5703574 ,  0.54912406],
       [-6.44746363,  1.71154942],
       [ 4.07117996, -2.09704393],
       [-6.98146485, -3.01121362],
       [-1.84359073,  7.77393895],
       [ 0.29912286, -1.67638128],
       [ 4.10627872,  0.19166195]])

In [67]:
print(pca.explained_variance_ratio_)
fig=px.bar(np.array(range(len(pca.explained_variance_ratio_))),np.array(pca.explained_variance_ratio_))
fig.show()

[0.49980097 0.1657685 ]


/opt/homebrew/Caskroom/miniforge/base/envs/trinit/lib/python3.10/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [68]:
wcss = []
for i in range(1, 10):
    kmeans = KMeans(n_clusters = i, init = "k-means++", max_iter = 500, n_init = 10, random_state = 123)
    kmeans.fit(data_pca)
    wcss.append(kmeans.inertia_)

print(wcss)

[347.427264023414, 133.26846843342727, 61.09781664768572, 36.14415185779761, 23.81218434915761, 13.888954401829018, 5.6974795646891625, 3.07777629698291, 1.2418346107139053]


In [69]:
fig = go.Figure(data = go.Scatter(x = [1,2,3,4,5,6,7,8,9,10], y = wcss))
fig.update_layout(title='WCSS vs. Cluster number',
                   xaxis_title='Clusters',
                   yaxis_title='WCSS')
fig.show()

/opt/homebrew/Caskroom/miniforge/base/envs/trinit/lib/python3.10/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [82]:
kmeans = KMeans(n_clusters = 2, init="k-means++", max_iter = 500, n_init = 10, random_state = 123)
identified_clusters = kmeans.fit_predict(data)


data_with_clusters = pd.DataFrame(data_pca,columns=["PCA1","PCA2"]).copy()
data_with_clusters['Cluster'] = identified_clusters

In [83]:
data_with_clusters

,PCA1,PCA2,Cluster
0,7.837482,-0.387927,0
1,-3.588127,-0.593836,1
2,-4.023775,-2.459873,1
3,6.570357,0.549124,0
4,-6.447464,1.711549,1
5,4.071180,-2.097044,0
6,-6.981465,-3.011214,1
7,-1.843591,7.773939,1
8,0.299123,-1.676381,1
9,4.106279,0.191662,0


In [92]:
fig = px.scatter(data_with_clusters, x = "PCA1", y="PCA2",
              color='Cluster', opacity = 0.8,size_max=30,template="plotly_dark")
fig.update_traces(marker={'size': 15})
fig.show()

/opt/homebrew/Caskroom/miniforge/base/envs/trinit/lib/python3.10/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

